In [37]:
import os
import torch
from models import resnet
from my_dataset import MyDataset
from my_lossfunc import AgentLoss, JointLoss, MultilabelLoss, DiscriminativeLoss
from my_transform import data_transforms
from torch.utils.data import DataLoader

In [38]:
# file & folder path
DATA_DIR_TRAIN = '../../Datasets/ReID_Market-1501/bounding_box_train'
DATA_DIR_TEST = '../../Datasets/ReID_Market-1501/bounding_box_test'
PKL_DIR_IN = 'storage/train_param.pkl'
PKL_DIR_OUT = 'storage/train_param_out.pkl'

In [39]:
BATCH_SIZE = 64
EPOCH = 30
LR = 0.1

In [40]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('Device:', DEVICE)

Device: cpu


## Prepare Work

In [41]:
# data loader
data_loader = {
    'train': DataLoader(
        dataset=MyDataset(DATA_DIR_TRAIN, 
                          mode='train', 
                          transform=data_transforms('train')),
        batch_size=BATCH_SIZE,
        shuffle=True,
    ),
    'val': DataLoader(
        dataset=MyDataset(DATA_DIR_TEST,
                          mode='test',
                          transform=data_transforms('test')),
        batch_size=BATCH_SIZE,
        shuffle=False,
    ),
}
print('data_loader: ok.')

data_loader: ok.


In [42]:
# neural network
net = resnet.get_model('resnet50_md')

if os.path.exists(PKL_DIR_IN):
    net.load_state_dict(torch.load(PKL_DIR_IN, map_location='cpu'))
    print('Parameters loaded to network.')
# else:
#     raise Exception('No parameters exists.')
net = net.to(DEVICE)

## Train

In [43]:
# loss functions
al_loss = torch.nn.CrossEntropyLoss().to(DEVICE)
rj_loss = JointLoss().to(DEVICE)  # lack 1 param
cml_loss = MultilabelLoss(BATCH_SIZE).to(DEVICE)
mdl_loss = DiscriminativeLoss().to(DEVICE)  # lack 1 param

In [ ]:
# optimizer
optimizer = torch.optim.SGD(lr=LR, momentum=0.9)